In [1]:
import pandas as pd
import re
import requests
import time
import numpy as np
import os

In [2]:
def clean_title(x):
    x = x.strip()
    x = re.sub('<br>', '', x)
    x = re.sub('\*', '', x)
    return x
def clean_authors(x):
    x = x.strip()
    x = re.sub('<br>', '', x)
    x = re.sub(r'\([^)]*\)', '', x)
    x = re.sub(r'[^\w\s,]', '', x)
    return x

def extract_publisher(x):
    x = x.split('.')[0]
    year = re.findall(r'\d{4}', x)
    publisher = x.replace(year[0], '').strip()
    return year[0], publisher

def extract_links_and_labels(text):
    # Regular expression to extract URLs and their corresponding labels
    regex_pattern = r'\[([^\[\]]+)\]\((https?://[^\s\)]+)\)'
    
    # Find all matches in the text
    matches = re.findall(regex_pattern, text)
    
    # Create a dictionary to store the links and labels
    links_and_labels = {}
    
    # Iterate through the matches and populate the dictionary
    for label, url in matches:
        links_and_labels[label] = url

    return links_and_labels


def get_repo_stars(repo_url):
    if repo_url is None:
        return None
    # Extract owner and repo name from the URL
    owner, repo = repo_url.split('/')[-2:]

    # Construct the API URL for getting repo information
    api_url = f'https://api.github.com/repos/{owner}/{repo}'

    time.sleep(np.random.uniform(0, 1))
    # Make GET request to the GitHub API
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse JSON response
        repo_info = response.json()
        # Extract and return the number of stars
        return repo_info['stargazers_count']
    else:
        # If request was not successful, print error message
        print(f"Failed to retrieve stars for {repo_url}. Status code: {response.status_code}")
        return None

In [3]:
if not os.path.exists('overview.csv'):
    path = 'inversion_techniques.txt'
    # Read lines 
    with open(path, 'r') as file:
        lines = file.readlines()

    # Indices of line breaks
    indices = [i for i, x in enumerate(lines) if x == '\n']
    indices = [0] + indices + [len(lines)]
    # Split lines into each paper
    papers = [lines[indices[i]:indices[i+1]] for i in range(len(indices)-1)]
    # Remove line breaks
    papers = [[l for l in elem if l != '\n'] for elem in papers]

    # Create overview table:
    data = []
    for paper in papers:
        title = clean_title(paper[0])
        authors = clean_authors(paper[1])
        year, publisher = extract_publisher(paper[2])
        links_and_labels = extract_links_and_labels(paper[2])
        code = links_and_labels.get('Code', None)
        project = links_and_labels.get('Project', None)
        pdf = links_and_labels.get('PDF', None)
        data.append([title, authors, publisher, year, code, project, pdf])
    df = pd.DataFrame(data, columns=['Title', 'Authors', 'Publisher', 'Year', 'Code', 'Project', 'PDF'])

    df['Github_Stars'] = df['Code'].apply(get_repo_stars)
    df.to_csv('overview.csv', index = False)
else: 
    df = pd.read_csv('overview.csv')
                      

Failed to retrieve stars for https://lvsn.github.io/OverparamStyleGAN/. Status code: 404
Failed to retrieve stars for https://rameenabdal.github.io/StyleFlow. Status code: 404
Failed to retrieve stars for https://github.com/davidbau/dissect/. Status code: 404
Failed to retrieve stars for https://github.com/RameenAbdal/image2stylegan-v2. Status code: 404


### Analysis

In [6]:
df.value_counts('Year').sort_index()

Year
2016     2
2017     3
2018     3
2019     5
2020    17
2021    21
2022    15
2023     3
Name: count, dtype: int64

In [8]:
df.Publisher.value_counts()

Publisher
CVPR                                         18
ECCV                                         11
ICCV                                         10
TOG                                           6
ICLR                                          3
IJCAI                                         2
AAAI                                          2
ICML                                          2
arxiv                                         2
CVPR  Workshop on AI for Content Creation     1
ACM MM                                        1
CVPR  (oral)                                  1
TIP                                           1
WACV                                          1
SIGGRAPH Asia                                 1
National Academy of Sciences                  1
IJCV                                          1
TNNLS                                         1
NeurIPS                                       1
AISTATS                                       1
ICLR  workshop                

In [14]:
print(f"Num without code: {len(df[df.Code.isna()])}")
df = df.dropna(subset=['Code'])

Num without code: 26


In [16]:
df.sort_values('Github_Stars', ascending=False).head(10)

,Title,Authors,Publisher,Year,Code,Project,PDF,Github_Stars
28,Encoding in Style: a StyleGAN Encoder for Imag...,"Elad Richardson, Yuval Alaluf, Or Patashnik, Y...",CVPR,2021,https://github.com/eladrich/pixel2style2pixel,None,https://arxiv.org/abs/2008.00951,3111.0
22,ReStyle: A Residual-Based StyleGAN Encoder via...,"Yuval Alaluf, Or Patashnik, Daniel CohenOr",ICCV,2021,https://github.com/yuval-alaluf/restyle-encoder,https://yuval-alaluf.github.io/restyle-encoder/,https://arxiv.org/abs/2104.02699,1022.0
46,StyleGAN2 Distillation for Feed-forward Image ...,"Yuri Viazovetskyi, Vladimir Ivashkin, Evgeny K...",ECCV,2020,https://github.com/EvgenyKashin/stylegan2-dist...,None,https://arxiv.org/abs/2003.03581,1007.0
15,HyperStyle: StyleGAN Inversion with HyperNetwo...,"Yuval Alaluf, Omer Tov, Ron Mokady, Rinon Gal,...",CVPR,2022,https://github.com/yuval-alaluf/hyperstyle,http://yuval-alaluf.github.io/hyperstyle/,https://arxiv.org/abs/2111.15666,992.0
37,e4e: Designing an Encoder for StyleGAN Image M...,"Omer Tov, Yuval Alaluf, Yotam Nitzan, Or Patas...",TOG,2021,https://github.com/omertov/encoder4editing,None,https://arxiv.org/abs/2102.02766,907.0
3,Pivotal Tuning for Latent-based Editing of Rea...,"Daniel Roich, Ron Mokady, Amit H Bermano, Dani...",TOG,2022,https://github.com/danielroich/PTI,None,https://arxiv.org/pdf/2106.05744.pdf,877.0
45,Rewriting a Deep Generative Model.,"David Bau, Steven Liu, Tongzhou Wang, JunYan Z...",ECCV,2020,https://github.com/davidbau/rewriting,None,https://arxiv.org/abs/2007.15646,536.0
48,Exploiting Deep Generative Prior for Versatile...,"Xingang Pan, Xiaohang Zhan, Bo Dai, Dahua Lin,...",ECCV,2020,https://github.com/XingangPan/deep-generative-...,None,https://arxiv.org/abs/2003.13659,474.0
47,In-Domain GAN Inversion for Real Image Editing.,"Jiapeng Zhu, Yujun Shen, Deli Zhao, Bolei Zhou",ECCV,2020,https://github.com/genforce/idinvert,https://genforce.github.io/idinvert/,https://arxiv.org/abs/2004.00049,457.0
13,High-Fidelity GAN Inversion for Image Attribut...,"Tengfei Wang, Yong Zhang, Yanbo Fan, Jue Wang,...",CVPR,2022,https://github.com/Tengfei-Wang/HFGI,https://tengfei-wang.github.io/HFGI/,https://arxiv.org/abs/2109.06590,449.0
